# COURSE: Master Python for scientific programming by solving projects
## PROJECT: Exploring cryptocurrency investments
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/maspy_x/?couponCode=202201

In [ ]:
# We need to install the crypto module:
!pip install Historic-Crypto

# library source
# https://pypi.org/project/Historic-Crypto/

In [ ]:
# the crypto functions
from Historic_Crypto import HistoricalData
from Historic_Crypto import Cryptocurrencies

# other stuff
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np

# Import and average data from one coin

In [ ]:
# Check the available assets
Cryptocurrencies().find_crypto_pairs()

In [ ]:
# import data for ethereum

# inputs to HistoricalData: coin, time segment in seconds, start-date (YYYY-MM-DD-HH-MM), end-date

eth = HistoricalData('ETH-EUR',60*60*24,'2020-01-01-00-00','2021-12-01-00-00').retrieve_data()
eth

In [ ]:
# Let's visualize some data
eth.plot.line()
# eth[['low','high','open','close']].plot.line()
# eth.plot.line(x='low',y='high',marker='o')

In [ ]:
# plot all pairs of variables
sns.pairplot(eth);

In [ ]:
# all price variables are really similar, so let's average them for convenience (also boosts SNR)

eth['price'] = eth[['low','high','open','close']].mean(axis=1)
eth

In [ ]:
# plot that over time
eth['price'].plot(logy=True);

# Create a dataframe of selected coins

In [ ]:
# Show all available indexes
Cryptocurrencies().find_crypto_pairs()

# other possibilities
Cryptocurrencies().find_crypto_pairs()['id'].tolist()

# maybe a bit easier:
for item in Cryptocurrencies().find_crypto_pairs()['id'].tolist():
  print(item)

In [ ]:
# Too many listings! Let's just print out the EUR ones

for item in Cryptocurrencies().find_crypto_pairs()['id'].tolist():
  if 'EUR' in item: print(item)

In [ ]:
# pick a few to focus on  (note: case-sensitive)
coins2eval = ['BTC-EUR','ETH-EUR','XLM-EUR','ALGO-EUR','LINK-EUR']
# 'ADA-EUR','DOGE-EUR','MATIC-EUR' # no available data from January 2020...


# initialize a dictionary to store the data
coinpricesD = {}

# extract the average price from each coin
for ticker in coins2eval:
  tmp = HistoricalData(ticker,60*60*24,'2020-01-01-00-00','2021-12-01-00-00',verbose=False).retrieve_data()
  coinpricesD[ticker] = tmp[['low','high','open','close']].mean(axis=1)

In [ ]:
# check out the dictionary...
coinpricesD

# ...then import to dataframe
coinprices = pd.DataFrame(coinpricesD)
coinprices

In [ ]:
# plot
coinprices.plot();

In [ ]:
# scale each column to a range of [0 1]
coinpricesScaled = (coinprices-coinprices.min()) / (coinprices.max()-coinprices.min())
coinpricesScaled = (coinprices-coinprices.min()) / (coinprices.loc['2021-05-11'].values[:]-coinprices.min())

# a simple plot
coinpricesScaled.plot(figsize=(15,8),
                      title='Scaled to 1.0 at 11 May 2021',fontsize=20);

# Data dimensionality via PCA

In [ ]:
# correlation matrix (same for original?)
coinpricesScaled.corr()

In [ ]:
# PCA
pca = PCA()
pca.fit(coinprices.dropna())
pcaS = PCA()
pcaS.fit(coinpricesScaled.dropna())


# plot the eigenspectra
plt.figure(figsize=(10,6))
plt.plot(100*pca.explained_variance_ratio_,'o-',markersize=10,label='Raw data')
plt.plot(100*pcaS.explained_variance_ratio_,'s-',markersize=10,label='Scaled data')

# make the plot look a bit nicer
plt.xticks(range(pca.n_components_))
plt.xlabel('Components')
plt.ylabel('Percent variance explained')
plt.legend()
plt.title('Scree plot of crypto time series data')
plt.show()

In [ ]:
# try PCA again with mean-subtraction

coinpricesScaledCentered = coinpricesScaled.sub(coinpricesScaled.mean(axis=1),axis=0)

coinpricesScaledCentered.plot(title='Mean-centered scaled data');


# PCA
pca = PCA()
pca.fit(coinpricesScaledCentered.dropna())

# repeat the plot
plt.figure(figsize=(10,7))
plt.plot(100*pca.explained_variance_ratio_,'o-',markersize=10)
plt.xticks(range(pca.n_components_))
plt.xlabel('Components')
plt.ylabel('Percent variance explained')
plt.title('Scree plot of crypto time series data')
plt.show()

# Simulating DCA investments

In [ ]:
## Simulation 1: 10 euros per day


# how much to invest per time unit (day)
dailyInvest = 10

# which coin to simulate
whichCoin = 'ETH-EUR'


# initialize investment
euroInvest = 0
coinInvest = 0


# loop through days
for dayi in range(coinprices.shape[0]):
  
  # how much crypto did we buy on this day?
  coin = dailyInvest / coinprices[whichCoin][dayi]

  # add to totals
  euroInvest += dailyInvest
  coinInvest += coin

# convert from coin to euro
eurosAtEnd = coinInvest*coinprices[whichCoin][-1]

# print out the results!
print(f'Total euro invested: \u20ac{euroInvest:,.2f}')
print(f'Total {whichCoin[:-4]} purchased: {coinInvest:.7f}')
print(f'End result: \u20ac{eurosAtEnd:,.2f}')

In [ ]:
## Simulation 2:  7 euros when the price is HIGHER than the previous day
#                15 euros when the price is LOWER than the previous day

dailyInvestUp =  7
dailyInvestDn = 15


# which coin to simulate
whichCoin = 'ETH-EUR'


# initialize investment
euroInvest = 0
coinInvest = 0


# loop through days
for dayi in range(1,coinprices.shape[0]):
  
  # is the coin up or down?
  if (coinprices[whichCoin][dayi] > coinprices[whichCoin][dayi-1]): # going up!
    coin = dailyInvestUp / coinprices[whichCoin][dayi]
    euroInvest += dailyInvestUp
  else: # going down!
    coin = dailyInvestDn / coinprices[whichCoin][dayi]
    euroInvest += dailyInvestDn

  # add to totals
  coinInvest += coin

# convert from coin to euro
eurosAtEnd = coinInvest*coinprices[whichCoin][-1]

# print out the results!
print(f'Total euro invested: \u20ac{euroInvest:,.2f}')
print(f'Total {whichCoin[:-4]} purchased: {coinInvest:.7f}')
print(f'End result: \u20ac{eurosAtEnd:,.2f}')

In [ ]:
## Simulation 3: 10 euros per day when price goes up
#                10*prct-diff when price goes down

dailyInvest = 10

# which coin to simulate
whichCoin = 'ETH-EUR'


# initialize investment
euroInvest = 0
coinInvest = 0

# initialize the percent change (not necessary but allows for visualization/exploration)
pctchng = [0]*coinprices.shape[0]


# loop through days
for dayi in range(1,coinprices.shape[0]):
  
  # percent change from previous day
  pctchng[dayi] = 100*(coinprices[whichCoin][dayi] - coinprices[whichCoin][dayi-1]) / coinprices[whichCoin][dayi-1]


  # determine investment amount according to relative price
  if pctchng[dayi]<0: # lower than previous day
    toinvest = dailyInvest * -pctchng[dayi]
  else: # equal or higher than previous day
    toinvest = dailyInvest
    
  # now add to totals
  coin = toinvest / coinprices[whichCoin][dayi]
  euroInvest += toinvest
  coinInvest += coin

# convert from coin to euro
eurosAtEnd = coinInvest*coinprices[whichCoin][-1]

# print out the results!
print(f'Total euro invested: \u20ac{euroInvest:,.2f}')
print(f'Total {whichCoin[:-4]} purchased: {coinInvest:.7f}')
print(f'End result: \u20ac{eurosAtEnd:,.2f}')

In [ ]:
toinvest,dailyInvest

In [ ]:
# histogram of percent changes
plt.hist(pctchng,bins=40);

# Bonus: Which coin should you have bought?

In [ ]:
# how much to invest per time unit (day)
dailyInvest = 10


# initialize dictionary of investments (coin:amount)
euroInvest = {}
coinInvest = {}


# loop through days
for dayi in range(coinprices.shape[0]):
  
  # loop over coins
  for coinname in coins2eval:
    
    # initialize on day 1
    if dayi==0:
      euroInvest[coinname] = 0
      coinInvest[coinname] = 0

    # how much crypto did we buy on this day?
    coin = dailyInvest / coinprices[coinname][dayi]

    # skip this data point if no data
    if np.isnan(coin): continue

    # add to totals
    euroInvest[coinname] += dailyInvest
    coinInvest[coinname] += coin


# convert from coin to euro and print
for coinname in coins2eval:
  eurosAtEnd = coinInvest[coinname]*coinprices[coinname][-1]
  print(f'{coinname[:-4]:>4}: \u20ac{euroInvest[coinname]:,.2f} \u21e8 \u20ac{eurosAtEnd:>9,.2f}')